In [2]:
import numpy as np
import pandas as pd
import random

In [3]:
fpath_uir = '../cornac/datasets/good_reads/good_read_UIR_1000.csv'
fpath_genres = '../cornac/datasets/good_reads/goodreads_genres.csv'
fpath_aspects = '../cornac/datasets/good_reads/uid_aspect_features.txt'

#read data
df = pd.read_csv(fpath_uir, sep='\t', header=0, names=['user_id', 'item_id', 'rating'])
#check user and item is included in item and user features
genres = pd.read_csv(fpath_genres)
item_features = np.array([[x,y] for [x,y] in zip(genres['item_id'].to_numpy(), genres['feature'].to_numpy())])
df = df[df['item_id'].isin(genres['item_id'])]
user_aspects = pd.read_csv(fpath_aspects, sep='\t', usecols=['user_id', 'feature'])
user_features = np.array([[x,y] for [x,y] in zip(user_aspects['user_id'].to_numpy(), user_aspects['feature'].to_numpy())])
df = df[df['user_id'].isin(user_aspects['user_id'])]

In [4]:
df_top100user = df.groupby('user_id').aggregate(['count']).reset_index().sort_values(('item_id', 'count'), ascending=False)[:100]
df_top100item = df.groupby('item_id').aggregate(['count']).reset_index().sort_values(('user_id', 'count'), ascending=False)[:100]

In [6]:
import random
def create_top_100(spacity):
    df = pd.DataFrame()
    items = df_top100item['item_id'].to_list()
    ratings = [1, 1.25, 1.5, 1.75, 2, 2.25, 2.5, 2.75, 3, 3.25, 3.5, 3.75, 4, 4.25, 4.5, 4.75, 5]
    for i in range(100):
        temp = pd.DataFrame({"user_id": [df_top100user['user_id'].tolist()[i] for _ in range(100)], "item_id": items, "rating": [random.choice(ratings) for _ in range(100)]})
        df = pd.concat([df, temp], ignore_index=True)

    #delete a fraction of data
    nums = int(len(df) * spacity)
    rows_to_delete = random.sample(range(len(df)), nums)
    df = df.drop(index=rows_to_delete)
    return df

In [7]:
df = create_top_100(0.1)

fpath_towrite = '../cornac/datasets/good_reads/good_read_dense.csv'
df.to_csv(path_or_buf = fpath_towrite, sep='\t', index=False)